# Functions to call for Speed Dating Experiment Data Set

# PART 1: CLEANING DATASET

### Function to drop data:

In [24]:
def dropData(main_df, remove_df): 
    main_df = main_df.drop(remove_df.index)  #removing old values
    return main_df

### Function to insert missing data to new df (fill in NaN values with 0)

In [29]:
#function to replace any NaN values with 0 where desired (takes in series; returns series)
def fillNaN(feature, df):
    df[feature] = df[feature].replace([np.nan], 0)

### Function for feature SETS: remove old values and replace with cleaned values

In [30]:
def cleanDF(main_df, df_all_feature_null, feature):
    atr_nan = main_df[main_df[feature].isnull()] #look where partner did not rate subject on feature
    atr_cleaned = atr_nan.drop(df_all_feature_null.index) #returning just rows with NaN data in feature but have other columns with data
    #amt = len(atr_cleaned) #test
    fillNaN([feature], atr_cleaned)  ##calling function fillNaN to replace NaN values with 0
    main_df = main_df.drop(atr_cleaned.index)  #removing old values
    main_df = pd.concat([main_df, atr_cleaned]) #adding cleaned data back into df
    return main_df

    
#main_df
#df_all_feature_null = all atr_o_NaN or all atr_NaN
#feature = looking for NaN values of this attribute
##atr_nan = df that holds 1 attribute with NaN values
##atr_cleaned = df that holds values where feature in question has NaN value but other attributes have ratings 
    #e.g. looking at attr_o; atr_fillNaN = rows that have NaN values for attr_o but have values for intel_o, since_o...etc


### Function to remove old NaN values and replace with new dataset of '0' values 

In [31]:
def cleanFeatDF(feature, main_df, cleaned_df): #clean feature subDF
    cleaned_df[feature] = cleaned_df[feature].replace([np.nan], 0)
    main_df = main_df.drop(cleaned_df.index)  #removing old values
    main_df = pd.concat([main_df, cleaned_df]) #adding cleaned data back into df
    return main_df


### Function to recount 'round' (AKA 'met_count')

Changes the value in 'round' to the number of people the person met with; takes into account subjects who's columns were dropped for bad data (i.e. if subject and partner didn't score each other than can assume they both didn't meet; e.g. value in 'match_ave' would be off since they met with less people than it's averaging a number out of)

In [32]:
def recountMET(df):
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        val = len(iidsubset)
        df.loc[df.iid == iid, 'round'] = [val]*val  #add it to a column in the original dataframe, not the iidsubset one -- for where the iid is equal to the one being calculated in your loop (hence the loc[] business)


******

**Function to check if lengths of dropna() values are the same**

If return 'FALSE':  all values are present or all are missing (i.e. subject didn't answer that question).
If return 'TRUE':  there are rows of data that are missng a value for the feature; e.g. person didn't want to rate low for other

In [4]:
def checkMissing (feature1, feature2, df):
    if (len(df[feature1].dropna())) != (len(df[feature2].dropna())):
        print True
    else:
        print False

        
        
        #UNCESSISARY LIKE THIS; better to iterate through 
        
        
        #checkAllMissing (df, feature_set)

****

# PART 2: CONDENSE DATASET

### Renaming Features:

In [5]:
def renameFeature(feature, new_feature_name, df):
    df.rename(columns={feature: new_feature_name}, inplace = True)


In [6]:
def renameFeatures(feature_key, new_feature_key, df):
    columns = df.columns
    new_columns = [row.replace(feature_key,new_feature_key) for row in columns]
    df.rename(columns=dict(zip(columns, new_columns)), inplace=True)

***Compress Features Within Dataset to Get Averages & Sums***

### Function for Sums:

Takes in a series (feature) & dataframe; Returns a series

In [7]:
def getSum(feature, df):
    list = []
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        if (iidsubset[feature].isnull().all()) == False: 
            sum = iidsubset[feature].sum()
            list.append(sum)
        else: 
            list.append(np.NaN)
    return pd.Series(list)



### Functions to Get Averages of a Feature Set:

Takes in a series (feature) & dataframe; Returns a series

#### Function to return **average score**:  
(take sum of values and divide by met_count)

*use when each row of data for the feature is different (within a subject's data)* 

In [8]:
def getAveScore(feature, df):
    list = []
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        if (iidsubset[feature].isnull().all()) == False:
            denom = iidsubset['met_count'].unique()[0]
            sum = iidsubset[feature].sum()
            divis = sum/denom
            list.append(divis)
        else: 
            list.append(np.NaN)
    return pd.Series(list)

In [ ]:
#EXAMPLE

#df_female_raw[['iid', 'attr_o', 'met_count']]
#get average scores of 'attr_o' for each iid 

#### Function to return **average of 1 rating / met_count** 

*use when each row of data for the feature is the same (within a subject's data)* 



In [9]:
def getAve(feature, df):
    list = []
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        if (iidsubset[feature].isnull().all()) == False:
            denom = iidsubset['met_count'].unique()[0] #returning 1 instance met_count
            nom = iidsubset[feature].unique()[0]  #returning 1 instance of the feature
            divis = nom/denom
            list.append(divis)
        else: 
            list.append(np.NaN)
    return pd.Series(list)

In [ ]:
#EXAMPLE

#df_female_raw[['iid', 'match_es', 'met_count']]
#want to get average of match_es (match_es/met_count)

#### Function to return expnum_ave (out of 20 people) and exphappy_ave (out of 10 pts):

Function takes in a feature, a value, and a dataframe and returns a series

In [10]:
def expAve(feature, denom, df):
    list = []
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        if (iidsubset[feature].isnull().all()) == False:
            f = iidsubset[feature].unique()[0]
            ave = f / denom
            list.append(ave)
        else:
            list.append(np.NaN)
    return pd.Series(list)

#EXPNUM_AVE (/20ppl)

#EXPHAPPY_AVE  (out of/10PTS)

### Function to Grab 1 Value of Feature Data for Each iid:
Takes in a series (feature) & dataframe; Returns a series

For features that have one consistent number for each person's iid #  (only want to return one instance of that number per iid #)

In [11]:
def getValueSet(feature, df):
    list = []
    for iid in df.iid.unique():
        iidsubset = df[df.iid == iid]
        if (iidsubset[feature].isnull().all()) == False:
            denom = len(iidsubset)
            sum = iidsubset[feature].sum()
            divis = sum/denom
            list.append(divis)
        else:
            list.append(np.NaN)
    return pd.Series(list)



In [ ]:
#test

#getValueSet('met_count', df_female_raw)

### Function to Convert Values from Old DF to New Condensed DF:

In [12]:
def ConvertDF(DF, df):  #DF = old dataframe    #df = new data frame
    df['iid'] = DF.iid.unique()  #returning iid# (1 unique value per person - 1 row per subject)
    df['gender'] = getValueSet('gender', DF) #returning 1 row per iid with subject's gender
    df['age'] = getValueSet('age', DF) #returning 1 row per iid with subject's age
    df['met_count'] = getValueSet('met_count', DF) #returning 1 row per iid with met_count info (how many people each person met with)
    df['exphappy'] = getValueSet('exphappy', DF) #returning rating for exphappy per iid 
    df['expnum'] = getValueSet('expnum', DF) #returning expnum per iid (1 value)
    df['match_es'] = getValueSet('match_es', DF) 
    
    df['attr_iMeasUp_1'] = getValueSet('attr_iMeasUp_1', DF)
    df['sinc_iMeasUp_1'] = getValueSet('sinc_iMeasUp_1', DF)
    df['intel_iMeasUp_1'] = getValueSet('intel_iMeasUp_1', DF)
    df['fun_iMeasUp_1'] = getValueSet('fun_iMeasUp_1', DF)
    df['amb_iMeasUp_1']= getValueSet('amb_iMeasUp_1', DF)
    
    df['attr_iMeasUp_2'] = getValueSet('attr_iMeasUp_2', DF)
    df['sinc_iMeasUp_2'] = getValueSet('sinc_iMeasUp_2', DF)
    df['intel_iMeasUp_2'] = getValueSet('intel_iMeasUp_2', DF)
    df['fun_iMeasUp_2'] = getValueSet('fun_iMeasUp_2', DF)
    df['amb_iMeasUp_2']= getValueSet('amb_iMeasUp_2', DF)
    
    df['attr_oPercveMe_1'] = getValueSet('attr_oPercveMe_1', DF)
    df['sinc_oPercveMe_1'] = getValueSet('sinc_oPercveMe_1', DF)
    df['intel_oPercveMe_1'] = getValueSet('intel_oPercveMe_1', DF)
    df['fun_oPercveMe_1'] = getValueSet('fun_oPercveMe_1', DF)
    df['amb_oPercveMe_1'] = getValueSet('amb_oPercveMe_1', DF)
    
    df['attr_oPercveMe_2'] = getValueSet('attr_oPercveMe_2', DF)
    df['sinc_oPercveMe_2'] = getValueSet('sinc_oPercveMe_2', DF)
    df['intel_oPercveMe_2'] = getValueSet('intel_oPercveMe_2', DF)
    df['fun_oPercveMe_2'] = getValueSet('fun_oPercveMe_2', DF)
    df['amb_oPercveMe_2'] = getValueSet('amb_oPercveMe_2', DF)
    
    df['attr_iRateMe_exp'] = getValueSet('attr_iRateMe_exp', DF)
    df['attr_iRateMe_exp'] = getValueSet('attr_iRateMe_exp', DF)
    df['intel_iRateMe_exp'] = getValueSet('intel_iRateMe_exp', DF)
    df['fun_iRateMe_exp'] = getValueSet('fun_iRateMe_exp', DF)
    df['amb_iRateMe_exp'] = getValueSet('amb_iRateMe_exp', DF)
    
    df['match_sum'] = getSum('match', DF)
    df['dec_sum'] = getSum('dec', DF) #sum of subject's decisions (num of 'yes'')
    df['dec_o_sum'] = getSum('dec_o', DF) #sum of parnter's decisions (num of 'yes'')
    
    df['match_es_ave'] = getAve('match_es', DF)  #MATCH_ES_AVE  = % of people they think they'll match with
    df['like_ave'] = getAveScore('like', DF)  #LIKE_AVE ##?
    df['like_o_ave'] = getAveScore('like_o', DF)  #LIKE_O_AVE ##?
    df['prob_ave'] = getAveScore('prob', DF)  #PROB_AVE ##?
    df['prob_o_ave'] = getAveScore('prob_o', DF)  #PROB_O_AVE ##?
       
    df['dec_ave'] = getAveScore('dec', DF)  #DEC_AVE (average decision of subject to want to date)
    df['dec_o_ave'] = getAveScore('dec_o', DF)  #DEC_AVE (average dec of partners to want to date subject)
    df['match_ave'] = getAveScore('match', DF) #MATCH_AVE (average match count/#of people met with)
    
    df['expnum_ave'] = expAve('expnum', 20, df)
    df['exphappy_ave'] = expAve('exphappy', 10, df)
    
    df['attr_ave'] = getAveScore('attr', DF) 
    df['sinc_ave'] = getAveScore('sinc', DF)  
    df['intel_ave'] = getAveScore('intel', DF) 
    df['fun_ave'] = getAveScore('fun', DF) 
    df['amb_ave'] = getAveScore('amb',  DF) 
    df['shar_ave'] = getAveScore('shar', DF) 
    
    df['attr_o_ave'] = getAveScore('attr_o', DF) 
    df['sinc_o_ave'] = getAveScore('sinc_o', DF)  
    df['intel_o_ave'] = getAveScore('intel_o', DF) 
    df['fun_o_ave'] = getAveScore('fun_o', DF) 
    df['amb_o_ave'] = getAveScore('amb_o', DF) 
    df['shar_o_ave'] = getAveScore('shar_o', DF) 
    
    return df